In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import math
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, SAGEConv, GATConv

from utils import *
from evaluation import evaluate

D:\Program Files (x86)\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
D:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [2]:
class Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_dim, noise_level=0.):
        super(Autoencoder, self).__init__()
        self.input_size, self.hidden_dim, self.noise_level = input_size, hidden_dim, noise_level
        self.fc1 = nn.Linear(self.input_size, self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim, self.input_size)
        
    def encoder(self, x):
        x = self.fc1(x)
        h1 = F.relu(x)
        return h1
    
    def mask(self, x):
        corrupted_x = x + self.noise_level * torch.randn_like(x)
        return corrupted_x
    
    def decoder(self, x):
        h2 = self.fc2(x)
        return h2
    
    def forward(self, x):
        out = self.mask(x)
        encode = self.encoder(out)
        decode = self.decoder(encode)
        return encode, decode

In [3]:
class GCN(torch.nn.Module):
    def __init__(self, features, hidden, classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(features, hidden)  # shape（输入的节点特征维度 * 中间隐藏层的维度）
        self.conv2 = GCNConv(hidden, classes)  # shape（中间隐藏层的维度 * 节点类别）

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x) 
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)# 输出每一节点的类别分布
    
    
class GraphSAGE(torch.nn.Module):
    def __init__(self, feature, hidden, classes):
        super(GraphSAGE, self).__init__()
        self.sage1 = SAGEConv(feature, hidden)
        self.sage2 = SAGEConv(hidden, classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.sage1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.sage2(x, edge_index)

        return F.log_softmax(x, dim=1)
    
    
class GAT(torch.nn.Module):
    def __init__(self, features, hidden, classes, heads=4):
        super(GAT, self).__init__()
        self.gat1 = GATConv(features, hidden, heads=heads)
        self.gat2 = GATConv(hidden*heads, classes) 

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.gat1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.gat2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [19]:
class Graph(torch.nn.Module):
    def __init__(self, data, output_size=8):
        super(Graph, self).__init__()
        self.x, self.edge_index = data.x, data.edge_index
        input_size = data.x.shape[1]
        hidden = int(input_size/2)
        self.sage1 = SAGEConv(input_size, hidden)
        self.sage2 = SAGEConv(hidden, output_size)
        self.embeddings = torch.zeros((self.x.shape[0], output_size), dtype=torch.float)

    def forward(self, x):
        if self.training:
            out = self.sage1(self.x, self.edge_index)
            out = F.relu(out)
            out = F.dropout(out, training=self.training)
            self.embeddings = self.sage2(out, self.edge_index)

        return self.embeddings[x]

In [46]:
class CoGa(torch.nn.Module):
    def __init__(self, rating_mat, data_u, data_i, embedding_size, hidden_dim, num_layers=2, nhead=8, dropout=0.0):
        super(CoGa, self).__init__()
        N, M = rating_mat.shape
        self.rating_mat = torch.from_numpy(rating_mat).float()
        self.graph_u, self.graph_i = Graph(data_u, embedding_size), Graph(data_i, embedding_size)
        self.ae_u, self.ae_i = Autoencoder(input_size=M, hidden_dim=embedding_size), Autoencoder(input_size=N, hidden_dim=embedding_size)
        encoder_layers = nn.TransformerEncoderLayer(d_model=2*embedding_size, nhead=nhead, dim_feedforward=hidden_dim, dropout=dropout)
        self.cell = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)
        self.linear = nn.Linear(2*2*embedding_size, 1)

    def forward(self, x):
        user_ids, item_ids = x[:,0], x[:,1]
        users = self.rating_mat[user_ids]
        items = self.rating_mat.t()[item_ids]
        ae_users, y_users = self.ae_u(users)
        ae_items, y_items = self.ae_i(items)
        graph_users, graph_items = self.graph_u(user_ids), self.graph_i(item_ids)
        embed_users, embed_items = torch.cat([ae_users, graph_users], 1), torch.cat([ae_items, graph_items], 1)
        out = torch.cat([embed_users, embed_items], 1).reshape(-1, 2, 2*embedding_size)
        out = self.cell(out)
        out = out.reshape(-1, 2*2*embedding_size)
        out = self.linear(out)
        return out, users, y_users, items, y_items
        

    def predict(self, pairs, batch_size, verbose):
        """Computes predictions for a given set of user-item pairs.
        Args:
          pairs: A pair of lists (users, items) of the same length.
          batch_size: unused.
          verbose: unused.
        Returns:
          predictions: A list of the same length as users and items, such that
          predictions[i] is the models prediction for (users[i], items[i]).
        """
        del batch_size, verbose
        num_examples = len(pairs[0])
        assert num_examples == len(pairs[1])
        predictions = np.empty(num_examples)
        pairs = np.array(pairs, dtype=np.int16)
        for i in range(num_examples):
            x = np.c_[pairs[0][i],pairs[1][i]]
            x = torch.from_numpy(x).long()
            out, _, _, _, _ = self.forward(x)
            predictions[i] = out.reshape(-1).data.numpy()
        return predictions

In [3]:
def co_mat(rating_mat):
    '''rating_mat: user-item interactoins'''
    N, M = rating_mat.shape
    co_u, co_i = np.zeros((N, N)), np.zeros((M, M))
    
    # obtain co-occurrence of users
    for i in range(N):
        for j in range(i+1, N):
            co_u[i,j] = np.sum(rating_mat[i,:]*rating_mat[j,:])
            co_u[j,i] = co_u[i,j]
    
    # obtain co-occurrence of items
    for i in range(M):
        for j in range(i+1, M):
            co_i[i,j] = np.sum(rating_mat[:,i]*rating_mat[:,j])
            co_i[j,i] = co_i[i,j]
    
    return co_u, co_i

In [4]:
def PMI_score(mat, k=2, is_normalised=False, eps=1e-20):
    '''mat: user-item interactoins'''
    N, M = mat.shape
    PMI_u, PMI_i = np.zeros((N, N)), np.zeros((M, M))
    NPMI_u, NPMI_i = np.zeros((N, N)), np.zeros((M, M))
    items, users = np.sum(mat,0), np.sum(mat,1)
    C_u, C_i = 0, 0   # the number of the co-occurrence user/item
    
    # obtain co-occurrence weights for users
    for i in range(N):
        for j in range(i+1, N):
            num_i, num_j = users[i], users[j]  # the number of the user i/j makes rates
            num_co = np.sum(mat[i,:]*mat[j,:]) # the number of users (i,j) make corate
            C_u += num_co
            PMI_u[i,j] = (num_co if num_co > 0 else eps) / (num_i * num_j if num_i * num_j > 0 else eps)
            PMI_u[j,i] = PMI_u[i,j]
    for i in range(N): PMI_u[i,i] = 1/C_u
    PMI_u = np.log2(PMI_u*C_u)
    
    # obtain co-occurrence weights for items
    for i in range(M):
        for j in range(i+1, M):
            num_i, num_j = items[i], items[j]  # the number of the item i/j rated by all users
            num_co = np.sum(mat[:,i]*mat[:,j]) # the number of items (i,j) corated by all users
            C_i += num_co
            PMI_i[i,j] = (num_co if num_co > 0 else eps) / (num_i * num_j if num_i * num_j > 0 else eps)
            PMI_i[j,i] = PMI_i[i,j]
    for i in range(M): PMI_i[i,i] = 1/C_i
    PMI_i = np.log2(PMI_i*C_i)
    
    PMI_u, PMI_i = PMI_u*(PMI_u > np.log2(k)), PMI_i*(PMI_i > np.log2(k))
    
    # NPMI
    if is_normalised:
        for i in range(N):
            for j in range(i+1, N):
                num_co = np.sum(mat[i,:]*mat[j,:]) # the number of users (i,j) make corate
                NPMI_u[i,j] = PMI_u[i,j]/(-np.log2(num_co/C_u if num_co>0 else eps))
                NPMI_u[j,i] = NPMI_u[i,j]

        for i in range(M):
            for j in range(i+1, M):
                num_co = np.sum(mat[:,i]*mat[:,j]) # the number of items (i,j) corated by all users
                NPMI_i[i,j] = PMI_i[i,j]/(-np.log2(num_co/C_i if num_co>0 else eps))
                NPMI_i[j,i] = NPMI_i[i,j]
                
        return NPMI_u, NPMI_i
    else:
        return PMI_u, PMI_i

In [ ]:
# 边的连接信息
# 注意，无向图的边要定义两次
edge_index = torch.tensor(
    [
        # 这里表示节点0和1有连接，因为是无向图
        # 那么1和0也有连接
        # 上下对应着看
        [0, 1, 1, 2],
        [1, 0, 2, 1],
    ],
    # 指定数据类型
    dtype=torch.long
)
# 节点的属性信息
x = torch.tensor(
    [
        # 三个节点
        # 每个节点的特征向量维度为1
        [-1],
        [0],
        [1],
    ]
)
edge_attr = 
data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

In [5]:
dataset_path = 'data/100k'

# Load the dataset
dataset = Dataset(dataset_path)
train_mat, test_ratings, test_negatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives
print('Dataset: #user=%d, #item=%d, #train_pairs=%d, #test_pairs=%d' 
      % (dataset.num_users, dataset.num_items, train_mat.nnz, len(test_ratings)))

Dataset: #user=943, #item=1682, #train_pairs=99057, #test_pairs=943


In [24]:
rating_mat = train_mat.todense()
mat = 1 * (rating_mat > 0)
print(type(mat) is np.matrix)
print(mat.shape)
print(mat)
print(mat[0])

True
(943, 1682)
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[1 0 0 ... 0 0 0]]


In [10]:
dataset = 'ml-100k'
data_dir = 'datasets/' + dataset + '/u.data'        # raw user-item records from the official website
N, M, data_list = load_data(file_dir=data_dir)

In [23]:
rating_mat = sequence2mat(data_list, N, M)
mat = 1 * (rating_mat > 0)
print(type(mat) is np.ndarray)
print(mat.shape)
print(mat)
print(mat[0])

True
(943, 1682)
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[1 0 0 ... 0 0 0]


In [17]:
co_u, co_i = co_mat(mat.A)

In [7]:
co_u[0]

array([ 0.,  3.,  9., 17.,  1.,  7.,  4.,  5., 19., 15., 16., 22., 10.,
       13.,  4., 28.,  3., 18., 14., 22., 12.,  4., 18., 18., 17.,  8.,
       14., 15.,  5.,  1., 15.,  1.,  6.,  6.,  5., 16.,  4., 22., 16.,
       12.,  4., 15.,  5., 12.,  2.,  8.,  8., 16.,  5.,  5.,  0., 10.,
       21.,  4., 17., 11.,  4.,  1., 30.,  5.,  8.,  6.,  8.,  6., 10.,
       22.,  1.,  8., 20.,  1., 19.,  9., 15.,  0.,  1., 24., 15.,  4.,
        1.,  2., 15., 10.,  6., 22.,  2., 12.,  4.,  8., 16.,  3., 21.,
       12.,  9., 14.,  5.,  9., 11., 16., 19.,  5., 19.,  5., 19., 11.,
        4., 14.,  2.,  4.,  2.,  6., 19., 13.,  1.,  2., 25.,  8., 12.,
        5., 19., 13., 11., 20.,  6.,  9.,  5.,  6., 19., 12.,  2., 20.,
        4.,  7.,  0.,  7.,  3.,  1.,  9., 17., 11.,  8.,  8.,  7.,  3.,
       12., 16.,  9.,  4.,  4.,  3.,  6.,  4., 10.,  0., 12., 14.,  0.,
        4.,  4., 11.,  6.,  0.,  4., 21.,  3.,  3.,  5.,  0., 12.,  4.,
       16.,  3.,  1.,  0.,  4.,  1.,  2.,  3.,  2.,  7.,  7.,  3

In [8]:
PMI_u, PMI_i = PMI_score(mat, k=2, is_normalised=False)

In [148]:
PMI_u[0]

array([ 0.        , 12.7699641 , 13.87848855, 13.90120863, 13.38663546,
       13.52723379, 13.18500159, 13.4305788 , 14.1620752 , 13.89435496,
       13.85058256, 13.99807017, 13.27560414, 14.36460915, 12.5690122 ,
       13.59705524, 13.36278871, 13.62582312, 13.30646511, 13.26110457,
       13.79368016, 13.60003909, 13.74920554, 13.24549336, 13.67708532,
       13.78970031, 13.76103097, 15.07629534, 13.12360105, 13.12360105,
       13.51216634, 12.95367605, 13.06470736, 14.31624613, 14.3580663 ,
       12.94369196, 15.18500159, 13.15118688, 13.05392752, 13.7699641 ,
       14.1539747 , 13.02299711, 14.07629534, 13.68619574, 14.38663546,
       13.37764667, 13.62110071, 13.61580641, 13.7637051 , 14.17251065,
       -0.        , 14.74508943, 13.52345215, 12.61053147, 14.0749272 ,
       13.65229533, 12.75436724, 12.12360105, 13.30257119, 14.3580663 ,
       13.00812383, 13.5656056 , 13.99431803, 12.74663159, 14.69064164,
       13.56808233, 13.18500159, 13.3956806 , 13.4305788 , 13.18

In [10]:
NPMI_u, NPMI_i = PMI_score(mat, k=4, is_normalised=True)

In [11]:
print(NPMI_i[0])

[0.         0.82128912 0.54555187 ... 0.70463701 0.70463701 0.70463701]


In [12]:
def get_edge_index(mat):
    N, M = mat.shape
    node0, node1 = [],[]
    for i in range(N):
        for j in range(M):
            if (mat[i,j] > 0) and (i != j):
                node0.append(i)
                node1.append(j)
    edge_index  = torch.tensor([node0, node1], dtype=torch.long)
    return edge_index

In [44]:
edge_index_u, edge_index_i = get_edge_index(NPMI_u), get_edge_index(NPMI_i)
x_u, x_i = torch.tensor(mat, dtype=torch.float), torch.tensor(mat.T, dtype=torch.float)
data_u, data_i = Data(x=x_u, edge_index=edge_index_u), Data(x=x_i, edge_index=edge_index_i)

In [ ]:
embedding_size = 8
hidden_dim = 16
epochs = 10
alpha = 0.1
batch_size = 1024
mode = 'hr'
topK = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CoGa(rating_mat, data_u, data_i, embedding_size, hidden_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.MSELoss() #nn.CrossEntropyLoss()

model.train()
for epoch in range(epochs):
    print('epoch: ', epoch)
    data_sequence = generate_instances(mat, positive_size=1, negative_time=4, is_sparse=False)
    data_array = np.array(data_sequence)
    x = torch.from_numpy(data_array[:,:2]).long()
    y = torch.from_numpy(data_array[:,-1]).reshape(-1,1)
    dataset = TensorDataset(x, y)
    data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
    for x, y in data_loader:
        print(batch_size)
        x, y = x.to(device), y.to(device)
        y_, u_, y_u, i_, y_i  = model(x)
        loss = criterion(y.float(), y_.float()) + alpha * (criterion(u_.float(), y_u.float()) + criterion(i_.float(), y_i.float()))
        optimizer.zero_grad()              # clear gradients for this training step
        loss.backward()                    # backpropagation, compute gradients
        optimizer.step()                   # apply gradients
    
    # Evaluation
    hr, ndcg = evaluate(model, test_ratings, test_negatives, topK)
    hr_list.append(hr)
    ndcg_list.append(ndcg)
    print('epoch=%d, loss=%.4f, HR=%.4f, NDCG=%.4f' %(epoch, loss, hr, ndcg))

    mlist = hr_list
    if mode == 'ndcg':
        mlist = ndcg_list
    if (len(mlist) > 10) and (mlist[-2] < mlist[-3] > mlist[-1]):
        best_hr, best_ndcg = hr_list[-3], ndcg_list[-3]
        break
    best_hr, best_ndcg = hr, ndcg

print("End. Best HR = %.4f, NDCG = %.4f. " %(best_hr, best_ndcg))

epoch:  0
1024


In [6]:
criterion = nn.MSELoss() #nn.CrossEntropyLoss()
aa = torch.tensor([0.1,1,0,0], dtype=torch.float)
bb = torch.tensor([0.1,0,1,1], dtype=torch.float)
criterion(aa,bb)

tensor(0.7500)

In [4]:
transformer_model = nn.Transformer(nhead=16, num_encoder_layers=12)
src = torch.rand((10, 32, 512))
tgt = torch.rand((20, 32, 512))
out = transformer_model(src, tgt)
print(out.shape)

torch.Size([20, 32, 512])


In [5]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
src = torch.rand(10, 32, 512)
out = transformer_encoder(src)
print(out.shape)

torch.Size([10, 32, 512])
